## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Omboue,GA,-1.5746,9.2618,78.58,85,100,5.48,overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,25.92,53,0,4.61,clear sky
2,2,Cape Town,ZA,-33.9258,18.4232,70.21,76,75,13.80,broken clouds
3,3,Fortuna,US,40.5982,-124.1573,55.47,73,0,20.71,clear sky
4,4,Nantucket,US,41.2835,-70.0995,25.02,46,0,9.22,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 98


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Omboue,GA,-1.5746,9.2618,78.58,85,100,5.48,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,78.62,67,53,10.92,broken clouds
11,11,Santa Rosa,PH,14.3122,121.1114,82.38,72,98,9.35,overcast clouds
12,12,Ternate,ID,0.8000,127.4000,82.15,77,44,4.14,scattered clouds
14,14,Kyabe,TD,9.4515,18.9449,78.62,15,71,5.37,broken clouds
22,22,Busselton,AU,-33.6500,115.3333,76.08,52,0,10.74,clear sky
23,23,Carutapera,BR,-1.1950,-46.0200,76.71,90,79,6.62,broken clouds
24,24,Broken Hill,AU,-31.9500,141.4333,80.31,32,100,4.61,overcast clouds
25,25,Puerto Ayora,EC,-0.7393,-90.3518,77.56,91,95,5.01,overcast clouds
26,26,Bengkulu,ID,-3.8004,102.2655,78.21,75,82,3.13,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                194
City                   194
Country                193
Lat                    194
Lng                    194
Max Temp               194
Humidity               194
Cloudiness             194
Wind Speed             194
Current Description    194
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Omboue,GA,78.58,overcast clouds,-1.5746,9.2618,
5,Rikitea,PF,78.62,broken clouds,-23.1203,-134.9692,
11,Santa Rosa,PH,82.38,overcast clouds,14.3122,121.1114,
12,Ternate,ID,82.15,scattered clouds,0.8000,127.4000,
14,Kyabe,TD,78.62,broken clouds,9.4515,18.9449,
22,Busselton,AU,76.08,clear sky,-33.6500,115.3333,
23,Carutapera,BR,76.71,broken clouds,-1.1950,-46.0200,
24,Broken Hill,AU,80.31,overcast clouds,-31.9500,141.4333,
25,Puerto Ayora,EC,77.56,overcast clouds,-0.7393,-90.3518,
26,Bengkulu,ID,78.21,broken clouds,-3.8004,102.2655,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotel_df = clean_hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Omboue,GA,78.58,overcast clouds,-1.5746,9.2618,Hotel Olako
5,Rikitea,PF,78.62,broken clouds,-23.1203,-134.9692,People ThankYou
11,Santa Rosa,PH,82.38,overcast clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home
12,Ternate,ID,82.15,scattered clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
14,Kyabe,TD,78.62,broken clouds,9.4515,18.9449,NaN
22,Busselton,AU,76.08,clear sky,-33.6500,115.3333,Observatory Guest House
23,Carutapera,BR,76.71,broken clouds,-1.1950,-46.0200,Lidera
24,Broken Hill,AU,80.31,overcast clouds,-31.9500,141.4333,Royal Exchange Hotel
25,Puerto Ayora,EC,77.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
26,Bengkulu,ID,78.21,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu


In [12]:
# drop NaN's
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Omboue,GA,78.58,overcast clouds,-1.5746,9.2618,Hotel Olako
5,Rikitea,PF,78.62,broken clouds,-23.1203,-134.9692,People ThankYou
11,Santa Rosa,PH,82.38,overcast clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home
12,Ternate,ID,82.15,scattered clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
22,Busselton,AU,76.08,clear sky,-33.6500,115.3333,Observatory Guest House
23,Carutapera,BR,76.71,broken clouds,-1.1950,-46.0200,Lidera
24,Broken Hill,AU,80.31,overcast clouds,-31.9500,141.4333,Royal Exchange Hotel
25,Puerto Ayora,EC,77.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
26,Bengkulu,ID,78.21,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
29,Carnarvon,AU,87.87,clear sky,-24.8667,113.6333,Hospitality Carnarvon


In [ ]:
#import numpy as np
# test before you apply
#clean_hotel_df_2 = clean_hotel_df.replace(r'^\s*$', np.nan, regex=True)
#clean_hotel_df_2.head(10)

In [ ]:
#clean_hotel_df_2 = clean_hotel_df_2.dropna()
#clean_hotel_df_2.head(10)

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City, Country</dt><dd>{City}, {Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
#what's wrong with info box content
from pprint import pprint 
info_box_content=hotel_info
pprint(info_box_content)

['\n'
 '<dl>\n'
 '<dt>Hotel Name</dt><dd>Hotel Olako</dd>\n'
 '<dt>City, Country</dt><dd>Omboue, GA</dd>\n'
 '<dt>Weather Description</dt><dd>overcast clouds</dd>\n'
 '<dt>Max Temp</dt><dd>78.58 °F</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Hotel Name</dt><dd>People ThankYou</dd>\n'
 '<dt>City, Country</dt><dd>Rikitea, PF</dd>\n'
 '<dt>Weather Description</dt><dd>broken clouds</dd>\n'
 '<dt>Max Temp</dt><dd>78.62 °F</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Hotel Name</dt><dd>Erlyn’s Sta Rosa City Home</dd>\n'
 '<dt>City, Country</dt><dd>Santa Rosa, PH</dd>\n'
 '<dt>Weather Description</dt><dd>overcast clouds</dd>\n'
 '<dt>Max Temp</dt><dd>82.38 °F</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Hotel Name</dt><dd>Sahid Bela Ternate Hotel</dd>\n'
 '<dt>City, Country</dt><dd>Ternate, ID</dd>\n'
 '<dt>Weather Description</dt><dd>scattered clouds</dd>\n'
 '<dt>Max Temp</dt><dd>82.15 °F</dd>\n'
 '</dl>\n',
 '\n'
 '<dl>\n'
 '<dt>Hotel Name</dt><dd>Observatory Guest House</dd>\n'
 '<dt>City, Countr

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
                             dissipating=False, max_intensity=300, point_radius=4)
# , info_box_content=hotel_info
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))